In [1]:
import pandas as pd
import numpy as np
import csv
import nltk
import random
import re
from nltk.stem import PorterStemmer
import sklearn
import sklearn.utils
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB

#nltk.download('punkt')
#nltk.download('stopwords')

In [2]:
tweets = pd.read_csv("tweets.csv", encoding='latin1', usecols=[0, 5], header=None, delimiter=',', quotechar='"')
tweets.columns = ['lable', 'text']
tweets.info()
tweets = sklearn.utils.shuffle(tweets, random_state=0)
tweets = tweets[:10000]
z = 0
o = 0
for n in tweets.lable:
    if(n == 0):
        z += 1
    else:
        o +=1

print("negative: ",z, "\t","Positive: ", o)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 2 columns):
lable    1600000 non-null int64
text     1600000 non-null object
dtypes: int64(1), object(1)
memory usage: 24.4+ MB
negative:  4954 	 Positive:  5046


In [6]:
def PreProcessDataset(Dataset):
    
    X = []
    sent_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    stopwords = nltk.corpus.stopwords.words('english')
    whitelist = ["n't", "not", "no"]
    Stemmer = PorterStemmer()
    for row in Dataset.text:
        row = re.sub(r'^https?:\/\/.*[\r\n]*', '', row, flags = re.MULTILINE) # Removing URLs
        row = re.sub(r'@\w+', '', row, flags = re.MULTILINE) # Removing accounts tag(@Ali)
        row = re.sub(r'[^\w\s]','', row) # Removing Punctuation
        row = sent_tokenizer.tokenize(row)
        tokens = []
        for sentence in row:
            words = nltk.word_tokenize(sentence)
            # Removing stopwords, Stemming and Converting every token to lowercase
            tokens += [Stemmer.stem(token.lower()) for token in words if (token.lower() not in stopwords or token in whitelist) and len(token) > 1]
        row = tokens
        X += [' '.join(row)]
        
    #print(X[:10])
    return X

In [7]:
def Extract_Features(tweets):
    vectorizer = TfidfVectorizer()
    
    features = vectorizer.fit_transform(tweets)
    return features

In [8]:
X = Extract_Features(PreProcessDataset(tweets))
y = tweets.lable
X_train, X_test, y_train, y_test  = train_test_split(X, y, test_size=0.20, random_state=47)

In [9]:
mnb = MultinomialNB()
mnb.fit(X_train, y_train)

score = mnb.score(X_test, y_test)

print("Model Acuuracy: %0.2f" % (score * 100), "%")
print("Number of mislabeled points out of a total %d points : %d" % (X_test.shape[0], (y_test != mnb.predict(X_test)).sum()))

Model Acuuracy: 74.15 %
Number of mislabeled points out of a total 2000 points : 517
